In [ ]:
# importation des bibliothèques
import requests
import pandas as pd
import sqlite3, sqlalchemy
from tqdm.notebook import trange, tqdm
from typing import List, Optional
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Float

# # IMDb files
# files = {
#     'titles': 'data/raw/title.basics.tsv.gz',
#     'ratings': 'data/raw/title.ratings.tsv.gz',
#     'crew': 'data/raw/title.principals.tsv.gz',
#     'people': 'data/raw/name.basics.tsv.gz',
#     'episodes': 'data/raw/title.episode.tsv.gz'
# }

# conn = sqlite3.connect('imdb.db')

# for table_name, file_path in files.items():
#     print(f"Loading {table_name}...")
#     df = pd.read_csv(file_path, 
#                      sep='\t', 
#                      na_values=['\\N'],
#                      low_memory=False)
#     df.to_sql(table_name, conn, if_exists='replace', index=False)
#     print(f"✓ {table_name}: {len(df)} rows")

# conn.close()

In [4]:
# création de la base de données newIMDB.db
engine = create_engine('sqlite:///data/db/newIMDB.db', echo=False)
meta = MetaData()

In [5]:
from sqlalchemy.engine import Engine

def load_table_from_tsv(
    engine: Engine,
    table,
    file_path: str,
    drop_columns: Optional[List[str]] = None,
    rename_columns: Optional[dict] = None,
    sep: str = "\t",
):
    """
    Generic loader for IMDb TSV tables into SQLite.

    Parameters
    ----------
    engine : sqlalchemy Engine
        SQLite engine
    table : sqlalchemy Table
        Target SQLAlchemy table definition
    file_path : str
        Path to .tsv or .tsv.gz file
    drop_columns : list[str], optional
        Columns to drop before insert
    rename_columns : dict, optional
        Column renaming mapping
    sep : str
        Field separator (default: tab)
    """

    # 1️⃣ Load TSV and convert \N → NULL
    df = pd.read_csv(
        file_path,
        sep=sep,
        na_values=["\\N"],
        low_memory=False
    )

    # 2️⃣ Optional transformations
    if drop_columns:
        for col in drop_columns:
            if col in df.columns:
                df.pop(col)

    if rename_columns:
        df.rename(columns=rename_columns, inplace=True)

    # 3️⃣ Convert dataframe → list of tuples
    values = df.to_records(index=False).tolist()

    if not values:
        return  # nothing to insert

    # 4️⃣ Create table if needed
    table.metadata.create_all(engine)

    # 5️⃣ Bulk insert with transaction
    with engine.connect() as connection:
        with connection.begin():
            markers = ",".join("?" * len(values[0]))
            sql = f"INSERT OR REPLACE INTO {table.name} VALUES ({markers})"
            connection.execute(sql, values)

    # 6️⃣ Free memory explicitly (important for large files)
    df = None
    values = None


In [6]:
from sqlalchemy import Table, Column, String, Integer, Float, MetaData

meta = MetaData()

ratings = Table(
    "ratings", meta,
    Column("title_id", String, primary_key=True),
    Column("rating", Float),
    Column("votes", String),
    extend_existing=True
)

crew = Table(
    "crew", meta,
    Column("title_id", String),
    Column("person_id", String),
    Column("category", String),
    Column("job", String),
    Column("characters", String)
)

episodes = Table(
    "episodes", meta,
    Column("episode_title_id", String),
    Column("show_title_id", String),
    Column("season_number", Integer),
    Column("episode_number", Integer)
)

titles = Table(
    "titles", meta,
    Column("title_id", String),
    Column("type", String),
    Column("primary_title", String),
    Column("original_title", String),
    Column("is_adult", Integer),
    Column("premiered", Integer),
    Column("ended", Integer),
    Column("runtime_minutes", Integer),
    Column("genres", String)
)

people = Table(
    "people", meta,
    Column("person_id", String),
    Column("name", String),
    Column("born", String),
    Column("died", String)
)


In [ ]:
load_table_from_tsv(
    engine,
    ratings,
    "data/raw/title.ratings.tsv.gz"
)


In [ ]:
load_table_from_tsv(
    engine,
    crew,
    "data/raw/title.principals.tsv.gz",
    drop_columns=["ordering"]
)


In [ ]:
load_table_from_tsv(
    engine,
    episodes,
    "title.episode.tsv.gz",
    rename_columns={
        "tconst": "episode_title_id",
        "parentTconst": "show_title_id"
    }
)


In [ ]:
load_table_from_tsv(
    engine,
    titles,
    "title.basics.tsv.gz",
    rename_columns={
        "tconst": "title_id",
        "titleType": "type",
        "primaryTitle": "primary_title",
        "originalTitle": "original_title",
        "startYear": "premiered",
        "endYear": "ended",
        "runtimeMinutes": "runtime_minutes"
    }
)


In [ ]:
load_table_from_tsv(
    engine,
    people,
    "name.basics.tsv.gz",
    rename_columns={
        "nconst": "person_id",
        "primaryName": "name",
        "birthYear": "born",
        "deathYear": "died"
    }
)


In [ ]:
################################################## title.ratings.tsv.gz ##################################################

# chargement des données
df_title_ratings = pd.read_csv(files['ratings'], compression='gzip', header=0, sep='\t')

# transformation des données du dataframe en liste de tuple
values = df_title_ratings.to_records(index=False).tolist()

# création de la table ratings
ratings = Table(
    'ratings', meta, 
    Column('title_id', String, primary_key=True), 
    Column('rating', Float), 
    Column('votes', String),
    extend_existing=True)

meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=ratings.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

# drop des data ratings
df_title_ratings = []
values = []

In [ ]:
################################################## title.principals.tsv.gz ##################################################

# chargement des données
df_title_principals = pd.read_csv(files['crew'], compression='gzip', header=0, sep='\t')

# suppresion d'une colonne
df_title_principals.pop('ordering')

# transformation des données du dataframe en liste de tuple
values = df_title_principals.to_records(index=False).tolist()

# création de la table ratings
crew = Table(
    'crew', meta, 
    Column('title_id', String, primary_key=False), 
    Column('person_id', String), 
    Column('category', String), 
    Column('job', String), 
    Column('characters', String) )
meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=crew.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_title_principals = []
values = []

In [ ]:
################################################## title.episode.tsv.gz ##################################################

# chargement des données
df_title_episode = pd.read_csv(files['episodes'], compression='gzip', header=0, sep='\t')

# transformation des données du dataframe en liste de tuple
values = df_title_episode.to_records(index=False).tolist()


# création de la table ratings
episodes = Table(
    'episodes', meta, 
    Column('episode_title_id', String, primary_key=False), 
    Column('show_title_id', String), 
    Column('season_number', Integer), 
    Column('episode_number', Integer))

meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=episodes.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_title_episode = []
values = []

In [ ]:
################################################## title.basics.tsv.gz ##################################################

# chargement des données
df_title_basics = pd.read_csv(files['titles'], compression='gzip', header=0, sep='\t')

# transformation des données du dataframe en liste de tuple
values = df_title_basics.to_records(index=False).tolist()


# création de la table ratings
titles = Table(
    'titles', meta, 
    Column('title_id', String, primary_key=False), 
    Column('type', String), 
    Column('primary_title', String), 
    Column('original_title', String), 
    Column('is_adult', Integer), 
    Column('premiered', Integer), 
    Column('ended', Integer), 
    Column('runtime_minutes', Integer), 
    Column('genres', String))

meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=titles.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_title_basics = []
values = []

In [ ]:
################################################## name.basics.tsv.gz ##################################################

# chargement des données
df_name_basics = pd.read_csv(files['names'], compression='gzip', header=0, sep='\t')

# suppresion d'une colonne
df_name_basics.pop('primaryProfession')
df_name_basics.pop('knownForTitles')

# transformation des données du dataframe en liste de tuple
values = df_name_basics.to_records(index=False).tolist()


# création de la table ratings
people = Table(
    'people', meta, 
    Column('person_id', String, primary_key=False), 
    Column('name', String), 
    Column('born', String), 
    Column('died', String))

meta.create_all(engine)

# insertions des valeurs dans la table ratings
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'éxécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT OR REPLACE INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=people.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en éxécutant la commande SQL
            connection.execute(ins, values)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
            
# drop des data ratings
df_name_basics = []
values = []

In [ ]:
# affichage du noms des tables dans la base de données newIMDB.db
inspector = inspect(engine)
inspector.get_table_names()